## <font color=blue> TASK 1: Install and setup LangChain
Welcome to this project notebook, which will serve as your guide to constructing your inaugural Generative AI application. Within this notebook, you'll encounter concise descriptions of each task to enhance your comprehension of the sequence. Our initial task commences with the importation of essential libraries required for this project.

***Note: Before importing the libraries please ensure that all the library modules such as Langchain, Streamlit, PyPdf and other required libraries are installed on this notebook by running the pip command as shown below.***

In [4]:
#For installing the Langchain module associated with OpenAI LLM model
!pip install langchain
!pip install langchain_openai
#For installing the Python library responsible for PDF upload
!pip install pypdf
#For installing the library responsible for web app development
!pip install streamlit
#For installing tokeniser library that asists with converting text strings into tokens recognizable by OpenAI models
!pip install tiktoken
#For installing the library used for invoking the environment file containing secret API key
!pip install python-dotenv
!pip install langchain_community

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 734.5 kB/s eta 0:00:0000:0100:01


In [5]:
import os
import dotenv
from langchain_openai import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import streamlit as st

## <font color=blue> Load OpenAI API Key to access LLM model

## <font color=black>
1. Go to https://platform.openai.com/api-keys
2. Click on the '+ Creat new secret key button'
3. Enter an identifier name(optional) and click on the "Create secret key" button
4. Copy the API key to be used in the API.env file that you need to upload to Google Colab environment

In [11]:
# Load the .env file and invoke the secret API key from the file
dotenv.load_dotenv('secretapi.env')
OpenAI.api_key = os.getenv("SECRET")

# <font color=blue> Load PDF file

In [14]:
pdf_url = "https://www.medrxiv.org/content/10.1101/2021.07.15.21260605v1.full.pdf"

loader = PyPDFLoader(pdf_url)
pages = loader.load_and_split()


In [15]:
#number of pages
print(len(pages))
print('--------------------------------------------------')


#view page content
print(pages[0].page_content)

18
--------------------------------------------------
COVID -19 Chest X-Ray Image  Classification  Using Deep Learning
Gunther Correia Bacellar,1 Mallikarjuna Chandrappa,1 Rajlakshman Kulkarni,1 Soumava Dey1* 
Department of Computer Science, University of Illinois at Urbana -Champaign, Urbana, IL, 61801, USA   
*Correspondence: soumava2@illinois.edu ; soumavadey87@gmail.com  
               
ABSTRACT  
The rise of the coronavirus disease 2019 (COVID -19) pandemic has made it necessary to improve existing medical screening 
and clinical management of this disease. While  COVID -19 patients are known to exhibit  a variety of symptoms, the major 
symptoms include  fever, cough, and fatigue. Since these symptoms also appear in pneumonia patients, this creates 
complications in COVID -19 detection especially during the flu season. Early studies identified abnormalities in chest X -ray 
images of COVID -19 infected patient s that could be beneficial for disease diagnosis. Therefore, chest X 

## <font color=blue> TASK 2: Define the summarize pdf function
Define the main function that will take pdf file path as an input and generate a summary of the file.

In [19]:
def summarize_pdf(pdf_file_path, chunk_size, chunk_overlap):

    #Instantiate LLM model gpt-3.5-turbo-16k
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, openai_api_key=OpenAI.api_key)
    #Load PDF file
    loader = PyPDFLoader(pdf_file_path)
    docs_raw = loader.load()

    #Create multiple documents
    docs_raw_text = [doc.page_content for doc in docs_raw]

    #Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs_chunks = text_splitter.create_documents(docs_raw_text)

    #Summarize the chunks (map_reduce method takes longer to execute)
    chain = load_summarize_chain(llm, chunk_type="stuff")
    
    #Return the summary
    summary = chain.invoke(docs_chunks, return_only_outputs=True)
    return summary['output_text']


In [20]:
#print summary by using chain type stuff or map_reduce
#Chunk size and chunk overlap values set to random value

print(summarize_pdf(pdf_url, 1000, 20))

ValidationError: 1 validation error for StuffDocumentsChain
chunk_type
  extra fields not permitted (type=value_error.extra)

## <font color=blue> TASK 3: Add Prompt template to the summarizer function
Leveraging prompt templates to extract key information from the reserach paper in more guided manner.

## <font color=blue> Define Prompt Templates

### <font color=black> Prompt Template for Stuffing chain type

In [10]:
map_prompt_template = """
                       Write a summary of the research paper for an
                       artficial intelligence researcher that includes
                       main points and any important details in bullet points.{text}
                      """

map_prompt =

### <font color=black> Add Combo Template for Map_Reduce chain type

In [14]:
combine_prompt_template =

combine_prompt =

In [15]:
#Modify the custom function to add the prompt templates

def summarize_pdf(pdf_file_path, chunk_size, chunk_overlap):

    #Instantiate LLM model gpt-3.5-turbo-16k
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, openai_api_key=OpenAI.api_key)

    #Load PDF file
    loader = PyPDFLoader(pdf_file_path)
    docs_raw = loader.load()

    #Create multiple documents
    docs_raw_text = [doc.page_content for doc in docs_raw]

    #Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs_chunks = text_splitter.create_documents(docs_raw_text)

    #Summarize the chunks
    chain = load_summarize_chain(llm, chain_type="stuff")


    #Return the summary
    summary = chain.invoke(docs_chunks, return_only_outputs=True)
    return summary['output_text']

In [ ]:
#print summary using the map_prompt and combine prompt
#Increasing the chunk size value might reduce the overall summarization time with map_reduce method
print(summarize_pdf(pdf_url, 2000, 100))

## <font color=blue> TASK 4: Build and test a GenAI app for PDF summarization

In [ ]:
%%writefile app.py
import os
import dotenv
from langchain_openai import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import streamlit as st


# Load the .env file and invoke the secret API key from the file
dotenv.load_dotenv('API.env')
OpenAI.api_key = os.getenv("OPEN_API_KEY")

#summarize_pdf function

def summarize_pdf(pdf_file_path, chunk_size, chunk_overlap, prompt):
    #Instantiate LLM model gpt-3.5-turbo-16k
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, openai_api_key=OpenAI.api_key)

    #Load PDF file
    loader = PyPDFLoader(pdf_file_path)
    docs_raw = loader.load()

    #Create multiple documents
    docs_raw_text = [doc.page_content for doc in docs_raw]

    #Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs_chunks = text_splitter.create_documents(docs_raw_text)

    #Summarize the chunks
    chain = load_summarize_chain(llm, chain_type="stuff", prompt = prompt)
    #Return the summary
    summary = chain.invoke(docs_chunks, return_only_outputs=True)
    return summary['output_text']

#streamlit app main() function

def main():
    #Set page config and title


    #Input pdf file path


    #prompt input

    #Summarize button


if __name__ == "__main__":
    main()

## <font color=blue> Launch Streamlit app from Google Colab

The following lines of code would enable users to launch Streamlit app from Google Colab using [ngrok service](https://ngrok.com/)

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [ ]:
!unzip ngrok-stable-linux-amd64.zip

In [22]:
get_ipython().system_raw('./ngrok http 8501 &')

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

## <font color=blue> FINAL TASK: Cumulative Activity

Click the link to explore useful Streamlit library functions:
https://docs.streamlit.io/library/cheatsheet

In [ ]:
%%writefile app.py
import os
import dotenv
from langchain_openai import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
import streamlit as st

# Load the .env file and invoke the secret API key from the file
dotenv.load_dotenv('API.env')
OpenAI.api_key = os.getenv("OPEN_API_KEY")

#summarize_pdf function

def summarize_pdf(pdf_file_path, chunk_size, chunk_overlap, chain_type, prompt):
    #Instantiate LLM model gpt-3.5-turbo-16k
    llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, openai_api_key=OpenAI.api_key)

    #Load PDF file
    loader = PyPDFLoader(pdf_file_path)
    docs_raw = loader.load()

    #Create multiple documents
    docs_raw_text = [doc.page_content for doc in docs_raw]

    #Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs_chunks = text_splitter.create_documents(docs_raw_text)

    #Create multiple prompts


    #Summarize the chunks

    #Return the summary
    return chain.run(docs_chunks)

#streamlit app main() function

def main():
    #Set page config and title
    st.set_page_config(page_title="PDF Summarizer", page_icon=":book:", layout="wide")
    st.title(" ")

    #Add custom sliders and selectbox for more user interaction


    #Display warning message


    #Input pdf file path
    pdf_file_path = st.text_input("Enter PDF file path:")

    #Prompt input
    user_prompt = st.text_input("Enter prompt:")

    #Summarize button
    if st.button("Summarize"):
        #Summarize pdf
        summary = summarize_pdf(pdf_file_path, chunk_size, chunk_overlap, chain_type, user_prompt)
        st.write(summary)

if __name__ == "__main__":
    main()

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501